PySpark tutorial
===
이 노트북에서는 Pyspark를 이용하여 SparkSQL을 실습해보기위해 실제 데이터셋을 가지고 데이터 분석을 진행해볼 것이다.<br>
목차는 아래와 같다
1. 배경 및 목적
2. 데이터 불러오기
3. EDA
4. 분석 및 가설 확인

## 배경 및 목적
### 문제정의
- 재구매율을 파악하여 사업모델 유형 파악
- Cohort 방식으로 유저의 행동 패턴을 파악하여 리텐션 탐색

### 가설
- 재구매율은 어떠한가? 몇 % 구간에 위치하는가? 고객의 충성도가 높은가? 혹은 신규고객을 발굴해야하는 유형인가?
- 당해년도 이탈률의 추이는 어떠한가? 코호트 별로 나아지고 있는가?

#### 세부 정의
1. 재구매율
- 재구매율은 아래와 같이 정의한다.
- 기준 기간은 당해년도를 기준으로 한다.
<b>$$재구매율 = \frac{재구매 고객 수}{전체 구매 고객 수} * 100$$</b>

2. 이탈률
- 이탈률의 정의는 사업 모델, 서비스 별로 다양하게 나타난다.
- 사용자 행동 데이터가 있으므로, 접속 여부를 기준으로 설정할 수 있다. 
- 사업 모델이 신규고객 확보인지, 고객 충성도 유형인지에 따라 기준 기간은 3일~30일 사이에서 결정할 예정이다.

## 데이터 불러오기
데이터는 캐글의 Fashion Campus 데이터를 사용한다. (링크 : https://www.kaggle.com/datasets/latifahhukma/fashion-campus)
- 데이터 크기: 4GB

### 데이터 확인

In [3]:
# Creating a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Spark_tutorial').getOrCreate()

In [4]:
# File path
path = r'C:\Users\Sunyoung_Jang\Documents\1.Dataset\Fashion_Campus\\'
file = 'click_stream.csv'

# Loading Data
df = spark.read.option('header','true').csv(path+file)
df.show(3)

+--------------------+----------+--------------------+--------------------+--------------+--------------+
|          session_id|event_name|          event_time|            event_id|traffic_source|event_metadata|
+--------------------+----------+--------------------+--------------------+--------------+--------------+
|fb0abf9e-fd1a-44d...|  HOMEPAGE|2019-09-06T15:54:...|9c4388c4-c95b-467...|        MOBILE|          null|
|fb0abf9e-fd1a-44d...|    SCROLL|2019-09-06T16:03:...|4690e1f5-3f99-42d...|        MOBILE|          null|
|7d440441-e67a-4d3...|  HOMEPAGE|2019-09-01T12:05:...|88aeaeb5-ec98-485...|        MOBILE|          null|
+--------------------+----------+--------------------+--------------------+--------------+--------------+
only showing top 3 rows



In [5]:
# Counting the data
print(f'# of rows : {df.count()}\n')

# of rows : 12833602



In [6]:
# Checking Schema
df.printSchema()  # I should've created a schema before loading an data

root
 |-- session_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- event_metadata: string (nullable = true)



### Creating schema
csv 데이터를 그냥 읽을 경우 아래와 같이 기본 데이터 타입이 string으로 설정되는 문제가 발생한다. 따라서 스키마를 사전에 정의하고 데이터를 읽어 효율적인 분석이 가능하도록 한다.
```
root
 |-- session_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- event_metadata: string (nullable = true)
```

**Tip!**<br>
미리 스키마를 정의하는 것은 3가지 장점이 있다.<br>

* 스파크가 데이터 타입을 추측해야 하는 책임을 덜어준다.
* 스파크가 스키마를 확정하기 위해 파일의 많은 부분을 읽어 들이려고 별도의 잡을 만드는 것을 방지한다. 데이터 파일이 큰 경우, 이는 비용과 시간이 많이 드는 작업이다.
* 데이터가 스키마와 맞지 않는 경우, 조기에 문제를 발견할 수 있다.

In [7]:
# 귀찮으니 함수를 쓰자
def load_data(schema, filename):
    return spark.read.option('header',True).schema(schema).csv(path+filename)

#### click stream
- 유저 로그 데이터를 보여준다.
- 이벤트 명, 트레픽 소스(모바일, PC 등), 구매 혹은 장바구니에 담았다면 상품명, 결제정보 등이 나타난다. 

In [8]:
filename = 'click_stream_new.csv'
schema = """`session_id` STRING
, `event_name` STRING 
, `event_time` TIMESTAMP
, `event_id` STRING NOT NULL
, `traffic_source` STRING
, `product_id` STRING
, `quantity` INTEGER
, `item_price` INTEGER
, `payment_status` STRING
, `search_keywords` STRING
"""

# Loading Data
click_stream = load_data(schema, filename)

click_stream.printSchema()
click_stream.limit(5).toPandas()

root
 |-- session_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_id: string (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_price: integer (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- search_keywords: string (nullable = true)



C:\Users\Sunyoung_Jang\anaconda3\envs\pyspark\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-07 00:54:32.821085,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,None,NaN,NaN,None,None
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-07 01:03:57.821085,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,None,NaN,NaN,None,None
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01 21:05:10.322763,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,None,NaN,NaN,None,None
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01 21:06:33.322763,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,15315.0,NaN,NaN,None,None
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01 21:15:29.425431,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,None,NaN,NaN,Success,None


#### customer
- 회원정보로, 기본적인 인적사항 및 성별, 생일, 기기 정보 등이 담겨있다.
- 아쉽게도 회원가입 일자는 없다.

In [9]:
filename = 'customer.csv'
schema = """`customer_id` STRING NOT NULL
, `first_name` STRING
, `last_name` STRING
, `username` STRING
, `email` STRING
, `gender` STRING
, `birthdate` DATE
, `device_type` STRING
, `device_id` STRING
, `device_version` STRING
, `first_join_date` DATE
"""

# Loading Data
customer = load_data(schema, filename)

customer.printSchema()
customer.show(5) 

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- username: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- device_type: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_version: string (nullable = true)
 |-- first_join_date: date (nullable = true)

+-----------+----------+-----------+--------------------+--------------------+------+----------+-----------+--------------------+--------------------+---------------+
|customer_id|first_name|  last_name|            username|               email|gender| birthdate|device_type|           device_id|      device_version|first_join_date|
+-----------+----------+-----------+--------------------+--------------------+------+----------+-----------+--------------------+--------------------+---------------+
|       2870|      Lala|    Maryati|67

#### product
- 제품의 타겟 성별, 시즌, 카테고리, 기본 색상, 패션스타일(캐쥬얼 등), 제품명을 보여준다.

In [10]:
filename = 'product.csv'
schema = """`id` STRING NOT NULL
, `gender` STRING
, `masterCategory` STRING
, `subCategory` STRING
, `articleType` STRING
, `baseColour` STRING
, `season` STRING
, `YEAR` INT
, `usage` STRING
, `productDisplayName` STRING
"""

# Loading Data
product = load_data(schema, filename)

product.printSchema()
product.show(5) 

root
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- masterCategory: string (nullable = true)
 |-- subCategory: string (nullable = true)
 |-- articleType: string (nullable = true)
 |-- baseColour: string (nullable = true)
 |-- season: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- usage: string (nullable = true)
 |-- productDisplayName: string (nullable = true)

+-----+------+--------------+-----------+-----------+----------+------+----+------+--------------------+
|   id|gender|masterCategory|subCategory|articleType|baseColour|season|YEAR| usage|  productDisplayName|
+-----+------+--------------+-----------+-----------+----------+------+----+------+--------------------+
|15970|   Men|       Apparel|    Topwear|     Shirts| Navy Blue|  Fall|2011|Casual|Turtle Check Men ...|
|39386|   Men|       Apparel| Bottomwear|      Jeans|      Blue|Summer|2012|Casual|Peter England Men...|
|59263| Women|   Accessories|    Watches|    Watches|    Silv

#### transaction
- 결제 정보를 보여준다. 
- session_id로 로그 데이터와 조인 가능
- 프로모션 및 배송 정보도 포함되어 있다.

In [11]:
filename = 'transaction_new.csv'
schema = """`created_at` DATE
, `customer_id` STRING
, `booking_id` STRING
, `session_id` STRING
, `payment_method` STRING
, `payment_status` STRING
, `promo_amount` INT
, `promo_code` STRING
, `shipment_fee` INT
, `shipment_date_limit` DATE
, `shipment_location_lat` STRING
, `shipment_location_long` STRING
, `total_amount` INT
, `product_id` STRING
, `quantity` INT
, `item_price` INT
"""

# Loading Data
transaction = load_data(schema, filename)

transaction.printSchema()
transaction.show(5) 

root
 |-- created_at: date (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- booking_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- promo_amount: integer (nullable = true)
 |-- promo_code: string (nullable = true)
 |-- shipment_fee: integer (nullable = true)
 |-- shipment_date_limit: date (nullable = true)
 |-- shipment_location_lat: string (nullable = true)
 |-- shipment_location_long: string (nullable = true)
 |-- total_amount: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_price: integer (nullable = true)

+----------+-----------+--------------------+--------------------+--------------+--------------+------------+-----------+------------+-------------------+---------------------+----------------------+------------+----------+--------+----------+
|created_at|customer_id|        

In [12]:
# 필요한 데이터만 골라내기
transaction = transaction.drop('shipment_date_limit','shipment_location_lat','shipment_location_long')
transaction.printSchema()

root
 |-- created_at: date (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- booking_id: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- promo_amount: integer (nullable = true)
 |-- promo_code: string (nullable = true)
 |-- shipment_fee: integer (nullable = true)
 |-- total_amount: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_price: integer (nullable = true)



## EDA 
- 결측값, 잘못된 데이터를 살펴보고 데이터 전처리 여부를 결정한다.

### 데이터 조회하는 두가지 방법
1. Spark Dataframe API를 활용하는 방법
- spark api를 알아야 한다(그런데 sql과 형태는 매우 비슷하다.)

2. Temp view 생성을 통해 SQL 쿼리를 돌리는 방법
- 스파크 SQL은 스키마를 가진 정형 데이터에서 ANSI SQL 2003 호환 쿼리를 사용할 수 있게 한다.
- Hive나 DB에서 데이터를 불러오지 않는 경우 세션별로 view를 생성해주어야한다는 특징
- 혹은 parquet 형태 데이터셋들을 파일로 저장하여 읽어들이는 방법도 있다. <br>
Link : https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#generic-loadsave-functions
<br>
<br>
두 방법 모두 스파크 쿼리는 논리/물리 계획 수립에서 최종 코드 생성까지 동일한 최적화 과정을 거치게 된다.
<br>

*Reference*. https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#run-sql-on-files-directly

In [13]:
# SoluAgg count for each event

from pyspark.sql.functions import count

avg_df = click_stream.groupBy('event_name').agg(count('event_name'))
avg_df.show()

+-----------+-----------------+
| event_name|count(event_name)|
+-----------+-----------------+
|     SCROLL|          1663571|
|ITEM_DETAIL|          1290396|
|  ADD_PROMO|           326535|
|    BOOKING|           852582|
|      CLICK|          2498038|
| PROMO_PAGE|           604931|
|ADD_TO_CART|          1937157|
|   HOMEPAGE|          2487126|
|     SEARCH|          1173266|
+-----------+-----------------+



In [14]:
# Register the DataFrame as a SQL temporary view
click_stream.createOrReplaceTempView("click_stream")

# Using spark.sql() 
spark.sql('SELECT event_name, COUNT(event_name) FROM click_stream GROUP BY event_name').show()

+-----------+-----------------+
| event_name|count(event_name)|
+-----------+-----------------+
|     SCROLL|          1663571|
|ITEM_DETAIL|          1290396|
|  ADD_PROMO|           326535|
|    BOOKING|           852582|
|      CLICK|          2498038|
| PROMO_PAGE|           604931|
|ADD_TO_CART|          1937157|
|   HOMEPAGE|          2487126|
|     SEARCH|          1173266|
+-----------+-----------------+



위에 보는 것 처럼, 스파크 SQL을 사용하여 SQL 쿼리를 실행할 수 있다. ANSI:2003과 호환되는 인터페이스가 제공되며, SQL과 데이터 프레임 간에 상호 운용성을 보여준다. (위 두 코드는 이벤트 별 발생 횟수를 보여주며, 두 실행 결과는 동일)

### 분석 대상 데이터 EDA
- 우리는 재구매율과 이탈률을 볼 것이다.
- 따라서 4가지 데이터 세트 중 click stream과 customer 데이터를 살펴볼 것이다.

#### 결측값 확인
- 귀찮은데 스킵할까 고민중

In [15]:
# TOTAL ROWS
spark.sql('SELECT COUNT(*) FROM click_stream').show()

+--------+
|count(1)|
+--------+
|12833602|
+--------+



In [16]:
# Null값 확인
from pyspark.sql.functions import isnan, when, count, col, isnull

click_stream.select([count(when(isnull(c), c)).alias(c) for c in click_stream.columns]).show()

+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|session_id|event_name|event_time|event_id|traffic_source|product_id|quantity|item_price|payment_status|search_keywords|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|         0|         0|         0|       0|             0|  10896445|12833602|  12833602|      11981020|       11660336|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+



**데이터프레임 API가 스파크 SQL보다 편리한 경우**<br>
위의 경우와 같이 어떠한 연산을 모든 컬럼에 적용하여 반복하는 경우 for문은 사용할 수 있다는 점에서 데이터프레임 API가 더 편리하다.

In [17]:
# Null 값 비중
import pyspark.sql.functions as f  # 내장함수와의 충돌 방지

click_stream.select([f.round((count(when(isnull(c), c))/12833602)*100, 2).alias(c) for c in click_stream.columns]).show()

+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|session_id|event_name|event_time|event_id|traffic_source|product_id|quantity|item_price|payment_status|search_keywords|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|       0.0|       0.0|       0.0|     0.0|           0.0|     84.91|   100.0|     100.0|         93.36|          90.86|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+



##### Error!
왜인지 모르겠지만 quantity와 item_price 데이터가 전부 null로 나타남. 데이터 원본은 문제가 없어 불러드리는 과정에서 망가진 것으로 추측된다.<br>
**알아낸 것**
- spark는 csv에서 파일을 읽을 때 ""과 그냥   이런 빈 값 둘다 null로 처리한다.
- python 에는 null 타입이 없다. (그래서 python 내장 메서드로 해결하려면 안된다)
- 숫자가 너무 크거나 (2147483647>) 숫자에 콤마가 포함 된 경우 string-> int는 null을 반환한다.
- Solution : string 타입으로 먼저 불러들인 후 데이터를 확인하여 변환한다.

In [18]:
filename = 'click_stream_new.csv'
schema = """`session_id` STRING
, `event_name` STRING 
, `event_time` TIMESTAMP
, `event_id` STRING NOT NULL
, `traffic_source` STRING
, `product_id` STRING
, `quantity` STRING
, `item_price` STRING
, `payment_status` STRING
, `search_keywords` STRING
"""

# Loading Data
click_stream = load_data(schema, filename)  # long 타입으로 64비트 인트형 사용하기

click_stream.printSchema()
click_stream['quantity', 'item_price'].show(5)  # 문제없이 데이터 출력됨

root
 |-- session_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_id: string (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_price: string (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- search_keywords: string (nullable = true)

+--------+----------+
|quantity|item_price|
+--------+----------+
|    null|      null|
|    null|      null|
|    null|      null|
|     4.0|  313529.0|
|    null|      null|
+--------+----------+
only showing top 5 rows



In [19]:
# 데이터 타입 변경

from pyspark.sql.types import FloatType, IntegerType

click_stream = click_stream.withColumn('quantity', click_stream.quantity.cast(IntegerType()))\
                .withColumn('item_price', click_stream.item_price.cast(IntegerType()))

click_stream['quantity', 'item_price'].show(5) 
click_stream.printSchema()

+--------+----------+
|quantity|item_price|
+--------+----------+
|    null|      null|
|    null|      null|
|    null|      null|
|       4|    313529|
|    null|      null|
+--------+----------+
only showing top 5 rows

root
 |-- session_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_id: string (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_price: integer (nullable = true)
 |-- payment_status: string (nullable = true)
 |-- search_keywords: string (nullable = true)



In [20]:
click_stream.filter('quantity is null').count()

10896445

**Solved!**<br>
String으로 먼저 정의하여 가져온 후 int형으로 변환하니 간단히 해결되었다. 다른 테이블의 경우 integer로 설정해도 null값만 출력되는 문제가 발생하지 않은 것은 `click_stream` 테이블만 csv상에 float형으로 설정되어서 에러가 나타났던 것으로 추측된다.

In [21]:
# Null 값 비중

click_stream.select([f.round((count(when(isnull(c), c))/12833602)*100, 2).alias(c) for c in click_stream.columns]).show()

+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|session_id|event_name|event_time|event_id|traffic_source|product_id|quantity|item_price|payment_status|search_keywords|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+
|       0.0|       0.0|       0.0|     0.0|           0.0|     84.91|   84.91|     84.91|         93.36|          90.86|
+----------+----------+----------+--------+--------------+----------+--------+----------+--------------+---------------+



#### Summary stats

In [22]:
# click_stream
for col in ['quantity','item_price']:
    click_stream.describe([col]).show()  # 이 작업은 수치형이 아닌 컬럼에 하면 상당히 오래걸린다.

+-------+------------------+
|summary|          quantity|
+-------+------------------+
|  count|           1937157|
|   mean| 1.439845092576389|
| stddev|1.4888804186634335|
|    min|                 1|
|    max|                48|
+-------+------------------+

+-------+------------------+
|summary|        item_price|
+-------+------------------+
|  count|           1937157|
|   mean|267640.70008935774|
| stddev|118162.00028964269|
|    min|              5600|
|    max|           1335959|
+-------+------------------+



#### Unique values of a categorical columns

In [23]:
# SQL 뷰 만들기
click_stream.createOrReplaceTempView('click_stream')

spark.sql("""SELECT COUNT(DISTINCT session_id) as session_id
                , COUNT(DISTINCT event_name) as event_name
                , COUNT(DISTINCT event_id) as event_id
                , COUNT(DISTINCT traffic_source) as traffic_source
                , COUNT(DISTINCT product_id)as product_id
                FROM click_stream""").show()
# 이것좀 이쁘게 보는 법 없나... : to_Pandas()
# Pivot 사용해보기
# 이 쿼리 진짜 오래걸림 2분

+----------+----------+--------+--------------+----------+
|session_id|event_name|event_id|traffic_source|product_id|
+----------+----------+--------+--------------+----------+
|    895203|         9|12833602|             2|     44446|
+----------+----------+--------+--------------+----------+



In [24]:
cnt_logs = spark.sql("""SELECT COUNT(DISTINCT quantity)
                    , COUNT(DISTINCT item_price)
                    , COUNT(DISTINCT payment_status)
                    , COUNT(DISTINCT search_keywords)
                    FROM click_stream""").show()

+------------------------+--------------------------+------------------------------+-------------------------------+
|count(DISTINCT quantity)|count(DISTINCT item_price)|count(DISTINCT payment_status)|count(DISTINCT search_keywords)|
+------------------------+--------------------------+------------------------------+-------------------------------+
|                      44|                    460872|                             2|                             23|
+------------------------+--------------------------+------------------------------+-------------------------------+



#### 데이터 기간은?
- 2016년 7월 ~ 2022년 8월 데이터이다.
- 따라서 가설 확인은 2022년도 8월을 기준으로 진행한다.

In [25]:
# 제일 최근 일자와 마지막 일자
spark.sql('SELECT MIN(event_time), MAX(event_time) FROM click_stream').show(1)

+--------------------+--------------------+
|     min(event_time)|     max(event_time)|
+--------------------+--------------------+
|2016-07-01 07:59:...|2022-08-01 09:49:...|
+--------------------+--------------------+



#### Event 종류는 어떻게 되는가?
- SCROLL : 스크롤의 깊이의 기준은 90%로 정의(전체 웹 페이지에서 몇 %까지인지는 데이터 명세가 없으므로, GA데이터로 가정)
- ITEM_DETAIL : 상세페이지 조회
- ADD_PROMO : 프로모션 페이지 클릭으로 유추된다.('할인 쿠폰 받기' 팝업을 클릭하는 것과 같이)
- PROMO_PAGE : 프로모션 페이지 클릭으로 유추된다.('할인 쿠폰 받기' 팝업을 클릭하는 것과 같이)

In [26]:
# 이벤트 종류 확인
spark.sql("""SELECT event_name, COUNT(event_name)
          FROM click_stream
          GROUP BY event_name""").show()

+-----------+-----------------+
| event_name|count(event_name)|
+-----------+-----------------+
|     SCROLL|          1663571|
|ITEM_DETAIL|          1290396|
|  ADD_PROMO|           326535|
|    BOOKING|           852582|
|      CLICK|          2498038|
| PROMO_PAGE|           604931|
|ADD_TO_CART|          1937157|
|   HOMEPAGE|          2487126|
|     SEARCH|          1173266|
+-----------+-----------------+



### 분석 1 : 사업 현황 분석

#### 재구매율 계산 (전자상거래 유형 확인)
린분석(2013)에 따르면 온라인 쇼핑몰이 고객과 어떤 종류의 관계를 맺고 있는지 아는 것은 매우 중요하며, 유형에 따라 마케팅 및 운영 전략이 달라진다고 언급한다. 이에 따라, 재구매율을 파악하여 아래 유형 중 어떤 유형에 속하는지 파악하고자 한다.

1. **신규고객 확보 유형**
    - 재구매율이 40% 이하
    - 포인트 제도가 도움 x
    - 마케팅 전략 : 제품 여러개 사게 만드는 것 x **구매자가 다른 사람들에게 이 쇼핑몰을 추천하도록 만드는 것**
    - Ex. 스쿠버 다이빙 장비, 암벽등반 장비 업체

<br>

2. **하이브리드 유형**
    - 재구매율 40%~60%
    - 신규고객, 재구매고객 둘 다를 바탕으로 회사가 성장, 둘 다 확보에 초점을 맞추어야
    - 사용자들은 연평균 2~2.5회 구매할 것
    - Ex. 자포스

<br>

3. **고객 충성도 유형**
    - 재구매율 60%
    - 충성도 높은 고객이 더 자주 구매하도록 만들어야 → 포인트 제도 효과적
    - Ex. 아마존

In [27]:
# 1. TOTAL # OF CUSTOMERS
customer.createOrReplaceTempView('customer')
transaction.createOrReplaceTempView('transaction')

spark.sql(""" SELECT COUNT(DISTINCT customer_id) Total_customers
                FROM customer""").show()

+---------------+
|Total_customers|
+---------------+
|         100000|
+---------------+



In [28]:
# 2. Reorder customers (2022)
spark.sql("""SELECT COUNT(DISTINCT c.customer_id) Total_reorder_customers
             FROM customer c
             INNER JOIN transaction t
             ON c.customer_id = t.customer_id
             WHERE YEAR(t.created_at) >= 2022""").show()

+-----------------------+
|Total_reorder_customers|
+-----------------------+
|                  32828|
+-----------------------+



In [29]:
# 3. Calculating the repeat order rate
print(f'Repeat order rate: {round(32828 / 100000 * 100, 2)}%')

Repeat order rate: 32.83%


2022년도 재구매율은 32.83%로 40% 이하이다. 따라서 전자상거래 유형은 신규고객 확보 유형임을 알 수 있다.

#### 22년도에 가입한 고객만을 대상으로 하는 재구매율은 어떠한가? 

In [30]:
# 1. TOTAL # OF CUSTOMERS WHO JOIN IN 2022
customer.createOrReplaceTempView('customer')
transaction.createOrReplaceTempView('transaction')

spark.sql(""" SELECT COUNT(DISTINCT customer_id) Total_customers
                FROM customer
                WHERE YEAR(first_join_date) == 2022""").show()

# 2. Reorder customers (2022)
spark.sql("""SELECT COUNT(DISTINCT c.customer_id) Total_reorder_customers
             FROM customer c
             INNER JOIN transaction t
             ON c.customer_id = t.customer_id
             WHERE YEAR(t.created_at) >= 2022
             AND YEAR(c.first_join_date) == 2022""").show()

+---------------+
|Total_customers|
+---------------+
|          14360|
+---------------+

+-----------------------+
|Total_reorder_customers|
+-----------------------+
|                   5912|
+-----------------------+



In [31]:
# 3. Reorder rate
print(f'Repeat order rate(2022): {round(5912 / 14360 * 100, 2)}%')

Repeat order rate(2022): 41.17%


2022년도 고객만을 대상으로 하는 재구매율은 xx%로 ~~ 유형에 속한다는 것을 알 수 있다.

#### 2022년 평균 DAU, MAU
- 데이터 한계 상 로그데이터에서 유저의 로그인 여부는 알 수 없다.
- 로그 상에서 Active user의 개수는 세션id의 개수로 정의한다.
- 즉, 겁나 뻥튀기된 값이라는 것...
- 그런데 실무에서도 이렇게 뻥튀기 된 값을 쓴다고 한다. 애초에 완벽하게 비로그인 유저의 수를 세는 것도 웹에서는 불가능 한 듯

In [32]:
# 2022 7월 평균 DAU 계산하기
query = """
SELECT 
    DATE(event_time), COUNT(session_id)
FROM 
    click_stream
WHERE 
    YEAR(event_time)==2022
AND 
    MONTH(event_time)==7
GROUP BY 
    DATE(event_time)
ORDER BY 
    DATE(event_time) DESC
"""

In [33]:
spark.sql(query).show()

+----------+-----------------+
|event_time|count(session_id)|
+----------+-----------------+
|2022-07-31|            14139|
|2022-07-30|             8542|
|2022-07-29|             8261|
|2022-07-28|            10540|
|2022-07-27|            10744|
|2022-07-26|            13957|
|2022-07-25|            14116|
|2022-07-24|            12919|
|2022-07-23|            11023|
|2022-07-22|            11694|
|2022-07-21|            12749|
|2022-07-20|            13424|
|2022-07-19|            13679|
|2022-07-18|            15634|
|2022-07-17|            15571|
|2022-07-16|            13052|
|2022-07-15|            13295|
|2022-07-14|            12946|
|2022-07-13|            12959|
|2022-07-12|            14240|
+----------+-----------------+
only showing top 20 rows



### 분석 2: 이탈률

## 결론